In [7]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.exc import SQLAlchemyError

def create_table_and_ingest_data(db_params, table_name, excel_file_path):
    try:
        # Read the Excel file into a DataFrame
        df = pd.read_excel(excel_file_path)
        print("Excel file read successfully.")
        print(df.head())  # Display the first few rows of the DataFrame for verification

        # Convert boolean columns to integers
        bool_cols = df.select_dtypes(include='bool').columns
        df[bool_cols] = df[bool_cols].astype(int)

        # Create the SQLAlchemy engine
        engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")

        # Reflect existing database schema to MetaData
        metadata = MetaData()
        metadata.reflect(bind=engine)

        # Check if the table exists in the reflected metadata
        if table_name in metadata.tables:
            print(f"Table '{table_name}' already exists.")
        else:
            # Define the table schema
            table = Table(table_name, metadata,
                          Column('id', Integer, primary_key=True),  # Example columns; adjust as per your schema
                          Column('name', String),
                          # Add other columns based on your Excel file
                          )

            # Create the table in the database
            metadata.create_all(engine)
            print(f"Table '{table_name}' created.")

        # Ingest data into the table
        df.to_sql(table_name, engine, if_exists='append', index=False)
        print("Data ingested successfully.")
    
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except SQLAlchemyError as e:
        print(f"An SQLAlchemy error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Database parameters
db_params = {
    'dbname': 'amazon',        # Replace with your actual database name
    'user': 'postgres',        # Replace with your actual username
    'password': '1234',        # Replace with your actual password
    'host': 'localhost',       # Adjust if your database server is not local
    'port': '5432'             # Default PostgreSQL port
}

# Define the table name and the path to the Excel file
table_name = 'amazon_sales'
excel_file_path = r'C:\Users\wissen\Downloads\Amazon Sales data 1.xlsx'

# Call the function to create the table and ingest data
create_table_and_ingest_data(db_params, table_name, excel_file_path)

Excel file read successfully.
                              Region                Country        Item Type  \
0              Australia and Oceania                 Tuvalu        Baby Food   
1  Central America and the Caribbean                Grenada           Cereal   
2                             Europe                 Russia  Office Supplies   
3                 Sub-Saharan Africa  Sao Tome and Principe           Fruits   
4                 Sub-Saharan Africa                 Rwanda  Office Supplies   

  Sales Channel Order Priority               Order Date   Order ID  Ship Date  \
0       Offline              H  2022-11-18 11:55:00 UTC  669165933 2010-06-27   
1        Online              C  2023-02-15 08:30:00 UTC  963881480 2012-09-15   
2       Offline              L  2023-01-10 10:45:00 UTC  341417157 2014-05-08   
3        Online              C  2023-03-20 14:20:00 UTC  514321792 2014-07-05   
4       Offline              L  2023-05-05 09:10:00 UTC  115456712 2013-02-06   

  